In [3]:
# -*- coding: utf-8 -*-
"""
Réseau de neurones récurrent, modèle de langue par mot, pour livres de Sherlock Holmes
Version avec couche vectorisation de mots et RNN
"""

import torch
torch.manual_seed(0) # Pour résultats reproductibles
from torch import nn
import pandas as pd
from collections import Counter
import re
taille_sequence = 8

# Déterminer si un GPU est disponible
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Entrainement sur ',device)

class DatasetSherlockHolmes(torch.utils.data.Dataset):
    """ Créer un Dataset avec les paroles de la colonne Lyric du fichier nom_fichier
    taille_sequence : taille d'une séquence de mots pour le modèle de langue
    Le texte est découpé en séquences de la taille taille_sequence
    """
    def __init__(self, taille_sequence=4,):
        self.taille_sequence = taille_sequence
        self.mots = self.charger_mots()
        self.mots_uniques = self.chercher_mots_uniques()

        self.index_a_mot = {index: mot for index, mot in enumerate(self.mots_uniques)}
        self.mot_a_index = {mot: index for index, mot in enumerate(self.mots_uniques)}

        self.mots_indexes = [self.mot_a_index[w] for w in self.mots]

    def charger_mots(self):
        with open('SherlockHolmes.txt') as f: 
            texte = f.read()
        return re.sub(r'[^\w\s]', '', texte).lower().split()

    def chercher_mots_uniques(self):
        frequence_mot = Counter(self.mots)
        return sorted(frequence_mot, key=frequence_mot.get, reverse=True)

    def __len__(self):
        return len(self.mots_indexes) - self.taille_sequence

    def __getitem__(self, index):
        return (
            torch.tensor(self.mots_indexes[index:index+self.taille_sequence]),
            torch.tensor(self.mots_indexes[index+1:index+self.taille_sequence+1]),
        )
    
class Modele(nn.Module):
    """Modèle de RNR avec une couche vectorisation, suivie d'une couche RNN et d'une couche linéaire"""
    def __init__(self, ds_paroles):
        super(Modele, self).__init__()
        self.taille_H_RNN = 128
        self.taille_vectorisation_mots = 64
        self.nombre_couches_RNR = 1

        taille_vocabulaire = len(ds_paroles.mots_uniques)
        self.vectorisation_mots = nn.Embedding(num_embeddings=taille_vocabulaire,
            embedding_dim=self.taille_vectorisation_mots)
        self.rnn = nn.RNN(input_size=self.taille_vectorisation_mots,hidden_size=self.taille_H_RNN,
            num_layers=self.nombre_couches_RNR,batch_first=True)
        self.dense_linaire = nn.Linear(self.taille_H_RNN, taille_vocabulaire)

    def forward(self, lot_X, etat_0):
        vectorisation = self.vectorisation_mots(lot_X)
        lot_Ht, etat = self.rnn(vectorisation, etat_0)
        lot_Yt = self.dense_linaire(lot_Ht)
        return lot_Yt, etat

    def initializer_etat(self, taille_sequence):
        return (torch.zeros(self.nombre_couches_RNR, taille_sequence, self.taille_H_RNN))

ds_paroles = DatasetSherlockHolmes(taille_sequence=taille_sequence)
modele = Modele(ds_paroles)
# Placer le modèle en mode GPU si possible
modele = modele.to(device)
    
import torch
import numpy as np
from torch import nn, optim
from torch.utils.data import DataLoader
import time
def entrainer_RNR(ds_paroles, modele, taille_lot=32, epochs=5, taille_sequence=6):
    debut = time.time()
    modele.train()
    dl_paroles = DataLoader(ds_paroles,batch_size=taille_lot)

    fonction_cout = nn.CrossEntropyLoss()
    optimizeur = optim.Adam(modele.parameters(), lr=0.001)

    for epoch in range(epochs):

        for lot, (lot_X, lot_Y) in enumerate(dl_paroles):
            lot_X = lot_X.to(device)
            lot_Y = lot_Y.to(device)
            etat = modele.initializer_etat(lot_X.shape[0])
            etat = etat.to(device)
            optimizeur.zero_grad()
            
            lot_Y_predictions, etat = modele(lot_X, etat)
            cout = fonction_cout(lot_Y_predictions.transpose(1, 2), lot_Y)
            
            cout.backward()
            optimizeur.step()
            if lot%100 == 0:
                print(f'-------- > epoch {epoch} lot {lot} :  coût = {cout.item()}')
                temps_ecoule = time.time() - debut
                print('Temps écoulé : {:.0f}m {:.0f}s'.format(temps_ecoule // 60, temps_ecoule % 60))


def predire(ds, modele, debut_texte, nb_mots=20, mode =0):
    """ Prédire une suite de nb_mots à partir de debut_texte selon le modele"""

    mots = debut_texte.split(' ')
    modele.eval()
    etat = modele.initializer_etat(1)
    etat = etat.to(device)
    for i in range(0, nb_mots):
        lot_X = torch.tensor([[ds.mot_a_index[m] for m in mots[i:]]])
        lot_X = lot_X.to(device)
        lot_Y_predictions, etat = modele(lot_X, etat)
        dernier_mot_Yt = lot_Y_predictions[0][-1]
        probs_dernier_mot = torch.nn.functional.softmax(dernier_mot_Yt, dim=0).data
        if mode == 0 :
            index_mot_choisi = torch.max(probs_dernier_mot, dim=0)[1].item()
        else :
            index_mot_choisi = torch.multinomial(probs_dernier_mot, 1)[0].item()
        mots.append(ds.index_a_mot[index_mot_choisi])
    return mots

entrainer_RNR(ds_paroles, modele, taille_lot=32, epochs=5, taille_sequence=taille_sequence)
print("Texte stochastique 1 :",predire(ds_paroles, modele, debut_texte='the adventure', mode = 1))
print("Texte stochastique 2 :",predire(ds_paroles, modele, debut_texte='the adventure', mode = 1))
print("Texte stochastique 3 :",predire(ds_paroles, modele, debut_texte='the adventure', mode = 1))
print("Texte max :",predire(ds_paroles, modele, debut_texte='the adventure', mode = 0))

Entrainement sur  cpu
-------- > epoch 0 lot 0 :  coût = 7.630110263824463
Temps écoulé : 0m 0s
-------- > epoch 0 lot 100 :  coût = 6.392582416534424
Temps écoulé : 0m 2s
-------- > epoch 0 lot 200 :  coût = 5.907230377197266
Temps écoulé : 0m 3s
-------- > epoch 1 lot 0 :  coût = 5.891152381896973
Temps écoulé : 0m 5s
-------- > epoch 1 lot 100 :  coût = 5.474283218383789
Temps écoulé : 0m 7s
-------- > epoch 1 lot 200 :  coût = 5.127549171447754
Temps écoulé : 0m 8s
-------- > epoch 2 lot 0 :  coût = 5.4953460693359375
Temps écoulé : 0m 10s
-------- > epoch 2 lot 100 :  coût = 4.972838401794434
Temps écoulé : 0m 11s
-------- > epoch 2 lot 200 :  coût = 4.634937763214111
Temps écoulé : 0m 13s
-------- > epoch 3 lot 0 :  coût = 5.055068492889404
Temps écoulé : 0m 14s
-------- > epoch 3 lot 100 :  coût = 4.5688157081604
Temps écoulé : 0m 16s
-------- > epoch 3 lot 200 :  coût = 4.249464988708496
Temps écoulé : 0m 18s
-------- > epoch 4 lot 0 :  coût = 4.594830513000488
Temps écoulé : 0

In [2]:
with open('SherlockHolmes.txt') as f: 
    texte = f.read()
len(texte)

610921

In [3]:
import re
re.sub(r'[^\w\s]', '', texte).lower().split()

['the',
 'adventures',
 'of',
 'sherlock',
 'holmes',
 'arthur',
 'conan',
 'doyle',
 'table',
 'of',
 'contents',
 'a',
 'scandal',
 'in',
 'bohemia',
 'the',
 'redheaded',
 'league',
 'a',
 'case',
 'of',
 'identity',
 'the',
 'boscombe',
 'valley',
 'mystery',
 'the',
 'five',
 'orange',
 'pips',
 'the',
 'man',
 'with',
 'the',
 'twisted',
 'lip',
 'the',
 'adventure',
 'of',
 'the',
 'blue',
 'carbuncle',
 'the',
 'adventure',
 'of',
 'the',
 'speckled',
 'band',
 'the',
 'adventure',
 'of',
 'the',
 'engineers',
 'thumb',
 'the',
 'adventure',
 'of',
 'the',
 'noble',
 'bachelor',
 'the',
 'adventure',
 'of',
 'the',
 'beryl',
 'coronet',
 'the',
 'adventure',
 'of',
 'the',
 'copper',
 'beeches',
 'a',
 'scandal',
 'in',
 'bohemia',
 'table',
 'of',
 'contents',
 'chapter',
 '1',
 'chapter',
 '2',
 'chapter',
 '3',
 'chapter',
 'i',
 'to',
 'sherlock',
 'holmes',
 'she',
 'is',
 'always',
 'the',
 'woman',
 'i',
 'have',
 'seldom',
 'heard',
 'him',
 'mention',
 'her',
 'under',

In [8]:
print(texte_concatene.lower().split())

NameError: name 'texte_concatene' is not defined

In [ ]:
texte_concatene = dataframe_entrainement[(dataframe_entrainement['language'] == 'en')].iloc[0:2]['Lyric'].str.cat(sep=' ')
re.sub(r'[^\w\s]', '', texte_concatene).lower().split()

In [ ]:
ds_paroles = DatasetSherlockHolmes(taille_sequence=taille_sequence)

In [ ]:
ds_paroles[0]